# Test of Intake connector on an example catalog

In [6]:
# all imports for this notebook
import intake
import satsearch
from satstac import ItemCollection
from lxml import objectify, etree

## Landsat-8 example catalog

In [7]:
landsat8_url = "https://raw.githubusercontent.com/sat-utils/sat-stac/master/test/catalog/catalog.json"
cat = intake.open_stac_catalog(landsat8_url)
list(cat)

['stac-catalog-eo']

### Retrieve catalog metadata

In [8]:
cat.metadata

{'id': 'stac-catalog',
 'description': 'An example STAC catalog',
 'stac_version': '1.0.0-beta.1'}

### List sub-catalogs

In [9]:
list(cat['stac-catalog-eo'])

['sentinel-s2-l1c', 'landsat-8-l1']

### Retrieve collections

In [10]:
ls8 = cat['stac-catalog-eo']['landsat-8-l1']
list(ls8)

['LC08_L1TP_152038_20200611_20200611_01_RT']

### Retrieve one item

In [11]:
ls8_item = ls8['LC08_L1TP_152038_20200611_20200611_01_RT']
list(ls8_item)

['ANG',
 'B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'B10',
 'B11',
 'BQA',
 'MTL',
 'thumbnail',
 'index']

### Retrieve item metadata

In [12]:
ls8_item.metadata

{'datetime': datetime.datetime(2020, 6, 11, 5, 54, 44, 650000, tzinfo=tzutc()),
 'gsd': 30,
 'platform': 'landsat-8',
 'instruments': ['oli', 'tirs'],
 'eo:cloud_cover': 6,
 'view:off_nadir': 0,
 'view:azimuth': 0,
 'view:sun_azimuth': 108.63187979,
 'view:sun_elevation': 68.84430949,
 'landsat:path': 107,
 'landsat:row': 18,
 'landsat:earth_sun_distance': 1.014156,
 'landsat:ground_control_points_verify': 114,
 'landsat:geometric_rmse_model': 7.562,
 'landsat:image_quality_tirs': 9,
 'landsat:ground_control_points_model': 313,
 'landsat:geometric_rmse_model_x': 5.96,
 'landsat:geometric_rmse_model_y': 4.654,
 'landsat:geometric_rmse_verify': 5.364,
 'landsat:image_quality_oli': 9,
 'bbox': [49.16354, 72.27502, 51.36812, 75.67662],
 'geometry': {'type': 'Polygon',
  'coordinates': [[[51.33855, 72.27502],
    [51.36812, 75.70821],
    [49.19092, 75.67662],
    [49.16354, 72.3964],
    [51.33855, 72.27502]]]},
 'date': datetime.date(2020, 6, 11),
 'catalog_dir': ''}

In [ ]:
cat.search('instruments')

## SAT-SEARCH 
Trying: Search functionality over STAC APIs: https://github.com/sat-utils/sat-search/blob/master/tutorial-1.ipynb

In [ ]:
URL =  "https://earth-search.aws.element84.com/v0"
results = satsearch.Search.search(url=URL,
                                bbox=[-110, 39.5, -105, 40.5],
                               datetime='2018-02-12T00:00:00Z/2018-03-18T12:31:12Z',
                               query=["eo:cloud_cover<10"],
                               collections=['sentinel-s2-l2a'])

print('Found %s items' % results.found())                                  
items = results.items(limit=5)
print('Summary: %s' % items.summary())

print('%s collections' % len(items._collections))
col = items._collections[0]
print('Collection: %s' % col)
#for key in col.keys():
#    print('%s: %s' % (key, col[key]))
    
print('\nITEMS: ')
for item in items:
    print('\nid: ',item.id)
    print('coll: ', item.collection())
    
    print("\nPROPERTIES:")
    print(item.properties.keys())
    for key in item.properties:
        print('%s: %s' % (key, item.properties[key]))
    
    print("\nASSETS: ")
    print(item.assets.keys())
    for key in item.assets.keys():
        print('%s: %s' % (key, item.assets[key]))
    break #just print first


In [ ]:
#save and load items collection to json
items.save('test.json')
items2 = ItemCollection.open('test.json')

print(items2.summary(['date', 'id', 'eo:cloud_cover']))

In [ ]:
# download a specific asset from all items and put in a directory by date in 'downloads'
filenames = items.download('metadata', filename_template='downloads/${id}')
print(filenames)

In [ ]:
# inspect a metadata file
input_file = 'downloads/S2B_12SXJ_20180316_1_L2A_metadata.xml'
root = objectify.parse(input_file)
print(etree.tostring(root))

## INTAKE-ESM
 data cataloging utility built on top of intake, pandas, and xarray

In [ ]:
col_url = "https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json"
col = intake.open_esm_datastore(col_url)
col

In [ ]:
col_subset = col.search(
         experiment_id=["historical", "ssp585"],
         table_id="Oyr",
         variable_id="o2",
         grid_label="gn",
)
col_subset